In [ ]:
import requests
import yaml
from time import sleep
from pprint import pprint

############
# Gathers all terms and borrowed Ontologies employed by Chemistry ontologies, listed in ontologies_details.yml
# var chem_ontos_terms - is dictionary holding ontology terms organized by ontology id/prefix
# var all_ontos - is dictionary where holding info on ontologies employed by chemestry ontologies, organized by ontology id/prefix
############



def yaml2dict(path):
    with open(path, 'r') as yaml_f:
        yaml_content = yaml_f.read()
        yaml_dict = yaml.safe_load(yaml_content)
    return yaml_dict

def get_all_ontology_terms(api_url, size, page, terms):
    api_url_w_params = f'{api_url}?page={page}&size={size}'
    # print(api_url_w_params)
    try:
        request = requests.get(api_url_w_params)
    except requests.exceptions.Timeout:
        sleep(1)
        request = requests.get(api_url_w_params)
    response = request.json()
    page_numb = response['page']['number']
    page_total  = response['page']['totalPages']
    # print(f'Current page:{page_numb}  Total pages:{page_total}')
    if '_embedded' in response.keys():
        terms.update({term['obo_id']: {'iri': term['iri'], 'id': term['obo_id'], 'onto_prefix': term['obo_id'].split(':')[0]} for term in response['_embedded']['terms'] if term['obo_id']})
        get_all_ontology_terms(api_url=api_url, size=size, page=page+1, terms=terms)
    return terms

def get_onto_info(onto_prefix):
    onto_api_url =  f'{tib_ols_api}ontologies/{onto_prefix}'
    onto_request = requests.get(onto_api_url)
    if onto_request.status_code == 404:
        onto_api_url =  f'{ebispot_ols_api}ontologies/{onto_prefix}'
        onto_request = requests.get(onto_api_url)
        if onto_request.status_code == 404:
            # print(f'No ontology info in OLS for {onto_prefix}')
            onto_response = {}     
        else:
            # print(f'getting {onto_prefix} from EBISPOT OLS')
            onto_response = onto_request.json() 
    else:
        # print(f'getting {onto_prefix} from TIB OLS')
        onto_response = onto_request.json() 
    return onto_response


tib_ols_api = 'https://service.tib.eu/ts4tib/api/'
ebispot_ols_api = 'http://www.ebi.ac.uk/ols/api/'
all_ontos = {} 
chem_ontos = yaml2dict(path='ontologies_details.yml')
chem_ontos_ids = [onto['id'].upper() for onto in chem_ontos]
chem_ontos_terms = {}
# for onto in ontos_used_in_ontos:
#     print(onto)
#     onto_response = get_onto_info(onto_prefix=onto)
#     pprint(onto_response)

for onto in chem_ontos:
    print(f"Ontology: {onto['id']}")
    all_onto_terms = get_all_ontology_terms(api_url=f'{tib_ols_api}ontologies/{onto["id"]}/terms', size=100, page=0, terms={})
    print(f"Ontology: {onto['id']} has {len(all_onto_terms)} terms")
    chem_ontos_terms[onto['id'].upper()] = all_onto_terms
    # pprint(all_onto_terms)

    ontos_used_in_ontos = set([v['onto_prefix'] for v in all_onto_terms.values()])
    print(f"Ontology: {onto['id']} uses the {', '.join(ontos_used_in_ontos)} ontologies")

    new_ontos_used_by_onto = {onto: get_onto_info(onto_prefix=onto) for onto in ontos_used_in_ontos if onto not in all_ontos.keys()}  # get info on ontology
    all_ontos.update(new_ontos_used_by_onto)  # & append to var all_ontos
    print(f"All ontologies: {all_ontos.keys()}")
    print(f'\n{"*"*20}\n')
    # pprint(all_ontos['PR'])

# pprint(chem_ontos_terms) # all terms for all chem ontologies listed in ontologies_details.yml
print(chem_ontos_terms.keys())

# enter missing chem_ontos instances into all_ontos
for chem_onto in chem_ontos_ids: 
    if chem_onto not in all_ontos.keys():
        print(chem_onto, "missing")
        new_entry = {chem_onto: {}}
        all_ontos.update(new_entry)


In [ ]:
# pprint(all_ontos['CDK'])
# print(all_ontos['CDK'].keys())
# print(all_ontos['RXNO']['config']['title'])
# all_ontos['CDK'].keys() is True

# chem_ontos_ids = [onto['id'].upper() for onto in chem_ontos]
# print(chem_ontos_ids)

# pprint(chem_ontos_terms['rxno'])
print('onto_dict', onto_dict.keys())
print(all_ontos.keys())
pprint(all_ontos['CDK'])
pprint(all_ontos['CHIRO'])
list(chem_ontos_terms.keys())[:1]

In [154]:
from pyvis.network import Network
target_onto = 'RXNO'
# for chem_onto in list(chem_ontos_terms.keys())[:1]:
net = Network(notebook=True, directed=True)
net.width = '1000px'
net.height = '750px'

# # all ontology nodes
# for onto, onto_dict in all_ontos.items():
#     if 'config' in onto_dict.keys():
#         onto_title = onto_dict['config']['title']
#     else:
#         onto_title = onto
#     if onto in chem_ontos_ids:
#         color='#ff0000'
#     else:
#         color='#ffff00'
#     net.add_node(onto, label=onto, title=onto_title,  color=color, borderWidth=0, shape="ellipse", size=500)

# for src_onto in chem_ontos_terms.keys():
#     # pass

net.add_node(target_onto, label=target_onto, title=target_onto,  color="#6BFFF5", borderWidth=0, shape="circle")

for i, (onto_term_k, dest_onto_term) in enumerate(chem_ontos_terms[target_onto].items()):
    if i:
        # add ontologies nodes
        onto =  dest_onto_term['onto_prefix']
        # onto_title = onto_dict[onto]['config']['title']
        # print(onto, onto_title)
        if onto in chem_ontos_ids:
            color='#C46BFF'
        else:
            color='#ffff00'
        net.add_node(onto, label=onto, title=onto,  color=color, borderWidth=0, shape="ellipse", size=500)


        # print(dest_onto_term['onto_prefix'], target_onto)
        if dest_onto_term['onto_prefix'] != target_onto:
            # print(src_onto, dest_onto_term['onto_prefix'])
            net.add_edge(target_onto, dest_onto_term['onto_prefix'], color='#595959', title=dest_onto_term['id'])
        # else: # how to represent own terms
        #     # net.add_node(dest_onto_term['id'],   color="#ff9999", borderWidth=0, shape="circle", size=5, group="selfterms")
        # net.add_edge(target_onto, dest_onto_term['onto_prefix'], color='#B2B3FF', title=dest_onto_term['id'], arrowStrikethrough=False)
net.repulsion(node_distance=400, central_gravity=0.15, spring_length=0.1, spring_strength=0.05, damping=0.1)
net.set_edge_smooth('dynamic')  # required for multiple edges to 1 node 
net.show(f'{target_onto}.html')

# issues to address:
# self terms removed: done
# unsused ontologies removed: done
# make target ontology different color/shape: done

# CHEBI needs to be included in ontologies_details.yml

In [164]:

target_onto = 'CHEMINF'
# for chem_onto in list(chem_ontos_terms.keys())[:1]:
# net = Network(notebook=True, directed=True)
net.width = '1000px'
net.height = '750px'

net.add_node(target_onto, label=target_onto, title=target_onto,  color="#6BFFF5", borderWidth=0, shape="circle")

for i, (onto_term_k, dest_onto_term) in enumerate(chem_ontos_terms[target_onto].items()):
    if i:
        # add ontologies nodes
        onto =  dest_onto_term['onto_prefix']
        # onto_title = onto_dict[onto]['config']['title']
        # print(onto, onto_title)
        if onto in chem_ontos_ids:
            color='#C46BFF'
        else:
            color='#ffff00'
        net.add_node(onto, label=onto, title=onto,  color=color, borderWidth=0, shape="ellipse", size=500)


        # print(dest_onto_term['onto_prefix'], target_onto)
        if dest_onto_term['onto_prefix'] != target_onto:
            # print(src_onto, dest_onto_term['onto_prefix'])
            net.add_edge(target_onto, dest_onto_term['onto_prefix'], color='#595959', title=dest_onto_term['id'])
        # else: # how to represent own terms
        #     # net.add_node(dest_onto_term['id'],   color="#ff9999", borderWidth=0, shape="circle", size=5, group="selfterms")
        # net.add_edge(target_onto, dest_onto_term['onto_prefix'], color='#B2B3FF', title=dest_onto_term['id'], arrowStrikethrough=False)
net.repulsion(node_distance=400, central_gravity=0.15, spring_length=0.1, spring_strength=0.05, damping=0.1)
net.set_edge_smooth('dynamic')  # required for multiple edges to 1 node 
net.show(f'{target_onto}.html')


In [156]:
# unuse functions

def get_term_info_from_src_onto(iri):
    term_page =  f'{tib_ols_api}terms/findByIdAndIsDefiningOntology?iri={iri}'
    try:
        termrequest = requests.get(term_page)
    except requests.exceptions.Timeout:
        sleep(1)
        termrequest = requests.get(term_page)

    termresponse = termrequest.json()
    if termresponse['page']['totalElements'] == 0:
        # QUERY ANOTHER N ANOTHER OLS INSTANCE if ontology not in TIB OLS
        term_page =  f'{ebispot_ols_api}terms/findByIdAndIsDefiningOntology?iri={iri}'
        termrequest = requests.get(term_page)
        termresponse = termrequest.json()    
    print(term_page)
    # pprint(termresponse)
    if '_embedded' in termresponse.keys():
        term = termresponse['_embedded']['terms'][0]
        term_info = {
            # 'api': term_api_url,
            'term_iri': iri,
            # 'label': apiresponse['label'],
            # '_id': apiresponse['annotation']['id'][0] if 'id' in apiresponse['annotation'] else None,
            # 'obo_id': apiresponse['obo_id']['id'] if type(apiresponse['obo_id']) is dict and 'id' in apiresponse['obo_id'].keys() else apiresponse['obo_id'],
            'onto_iri': term['ontology_iri'],
            'onto_prefix': term['ontology_prefix'],
            'onto_name': term['ontology_name'],
            }
        return term_info
    else:
        return None



def terms_hierarchy(api_url):
    # calls OLS API on ontology terms 
    # and requests the terms children, recursively 
    apirequest = requests.get(api_url)
    apiresponse = apirequest.json() 
    if '_embedded' in apiresponse.keys():
        terms = apiresponse['_embedded']['terms']
        for term in terms:
            # pprint(term)
            term_iri = term['iri']
            print(f'\nterm_iri: {term_iri}')
            # ols_term_url = term['_links']['self']['href']
            # print(ols_term_url)
            term_info =get_term_info_from_src_onto(iri=term_iri)

            pprint(term_info)
            # if 'children' in term['_links'].keys():
            #     api_children_url = term['_links']['children']['href']
            #     terms_hierarchy(api_url=term['_links']['children']['href'])
            